# LegiScan Datasets

LegiScan very nicely provides well-formatted sources for all bills considered by state legislature.

I've downloaded the TN House ones into a subfolder and this notebook parses out the relevant information.

I manually merged the `people` directories from each session into a single one with all legislators from all sessions.

In [79]:
import pandas as pd
from pathlib import Path

In [80]:
# This folder has sub-folders for each legislative session
# as downloaded from LegiScan
root = Path('tn-house-bills')

In [81]:
people = pd.concat(
    [
        pd.read_json(person_json)
        for person_json in (root / 'people').glob('*.json')
    ],
    axis=1).T.set_index('people_id')[['name', 'last_name', 'party', 'role']]
people.head()

,name,last_name,party,role
people_id,,,,
7298,Jack Johnson,Johnson,R,Sen
7277,Diane Black,Black,R,Sen
12161,Don Miller,Miller,R,Rep
7220,Jeanne Richardson,Richardson,D,Rep
18645,Mark Lovell,Lovell,R,Rep


In [82]:
bills = pd.concat([
    pd.read_json(bill_json) for year_dir in root.glob('*General_Assembly')
    for bill_json in (year_dir / 'bill').glob('*.json')
    if year_dir.name != 'people' and year_dir.is_dir()
],
                  axis=1).T.set_index('bill_id')[[
                      'bill_number', 'bill_type', 'body', 'completed',
                      'description', 'session', 'sponsors', 'status',
                      'subjects', 'url', 'votes'
                  ]]
bills['subjects'] = [
    '; '.join([subj['subject_name'] for subj in subjects])
    for subjects in bills.subjects
]  # all bills actually appear to only have one subject
bills['category'] = bills['subjects'].str.split(', ').str[0]
bills.head()

,bill_number,bill_type,body,completed,description,session,sponsors,status,subjects,url,votes,category
bill_id,,,,,,,,,,,,
1078171,SB2524,B,S,1,"As enacted, authorizes utility districts to en...","{'session_id': 1428, 'state_id': 42, 'year_sta...","[{'people_id': 7274, 'person_hash': '9qzw4udj'...",4,"Utilities, Utility Districts",https://legiscan.com/TN/bill/SB2524/2017,"[{'roll_call_id': 723064, 'date': '2018-03-13'...",Utilities
948125,SB0519,B,S,0,"As introduced, enacts the ""Tennessee Retail Pe...","{'session_id': 1428, 'state_id': 42, 'year_sta...","[{'people_id': 13706, 'person_hash': 'nnuu3olz...",2,Pets,https://legiscan.com/TN/bill/SB0519/2017,"[{'roll_call_id': 598336, 'date': '2017-03-06'...",Pets
943606,HJR0084,JR,H,1,"Seth T. and Charlotte Bible, 50th wedding anni...","{'session_id': 1428, 'state_id': 42, 'year_sta...","[{'people_id': 12069, 'person_hash': 'xv3b124h...",4,"Memorials, Personal Occasion",https://legiscan.com/TN/bill/HJR0084/2017,"[{'roll_call_id': 577995, 'date': '2017-02-06'...",Memorials
1077899,HB2402,B,H,0,"As introduced, requires the commissioner to pr...","{'session_id': 1428, 'state_id': 42, 'year_sta...","[{'people_id': 18636, 'person_hash': 'd07h6eqg...",1,"Commerce and Insurance, Dept. of",https://legiscan.com/TN/bill/HB2402/2017,[],Commerce and Insurance
1075969,SB2174,B,S,1,"As enacted, establishes requirements for issua...","{'session_id': 1428, 'state_id': 42, 'year_sta...","[{'people_id': 15132, 'person_hash': '661zfjyf...",4,"Education, Higher",https://legiscan.com/TN/bill/SB2174/2017,"[{'roll_call_id': 719163, 'date': '2018-03-07'...",Education


In [121]:
# Looking for just the last vote per bill
votes = pd.concat(
    [
        pd.read_json(vote_json) for year_dir in root.glob('*General_Assembly')
        for vote_json in (year_dir / 'vote').glob('*.json')
        if year_dir.name != 'people' and year_dir.is_dir()
    ],
    axis=1).T.pipe(lambda df: df[df['chamber'] == 'H']).sort_values(
        'date').groupby('bill_id').last()

In [122]:
# Karen Camper is our candidate of interest


def get_camper_vote(votes):
    for vote in votes:
        if vote['people_id'] == 7242:
            return vote['vote_text']
    return 'no record'


votes['camper_vote'] = votes.votes.apply(get_camper_vote)

In [123]:
combined = bills.join(votes[['camper_vote', 'yea', 'nay', 'total']])
combined['camper_sponsored'] = [
    any(sponsor['people_id'] == 7242 for sponsor in sponsor_set)
    for sponsor_set in bills['sponsors']
]
combined = combined.drop(
    columns=['votes', 'sponsors', 'completed']).fillna('no record')
combined['session'] = combined.session.str['session_title']
combined['status'] = combined['status'].replace({
    1: 'introduced',
    2: 'engrossed',
    3: 'enrolled',
    4: 'passed',
    6: 'failed'
})

In [129]:
combined.to_csv('tn-house-bills.csv')

In [124]:
combined

,bill_number,bill_type,body,description,session,status,subjects,url,category,camper_vote,yea,nay,total,camper_sponsored
bill_id,,,,,,,,,,,,,,
1078171,SB2524,B,S,"As enacted, authorizes utility districts to en...",2017-2018 Regular Session,passed,"Utilities, Utility Districts",https://legiscan.com/TN/bill/SB2524/2017,Utilities,Yea,89.0,0.0,89.0,False
948125,SB0519,B,S,"As introduced, enacts the ""Tennessee Retail Pe...",2017-2018 Regular Session,engrossed,Pets,https://legiscan.com/TN/bill/SB0519/2017,Pets,Nay,45.0,47.0,94.0,False
943606,HJR0084,JR,H,"Seth T. and Charlotte Bible, 50th wedding anni...",2017-2018 Regular Session,passed,"Memorials, Personal Occasion",https://legiscan.com/TN/bill/HJR0084/2017,Memorials,Yea,96.0,0.0,97.0,False
1077899,HB2402,B,H,"As introduced, requires the commissioner to pr...",2017-2018 Regular Session,introduced,"Commerce and Insurance, Dept. of",https://legiscan.com/TN/bill/HB2402/2017,Commerce and Insurance,no record,no record,no record,no record,False
1075969,SB2174,B,S,"As enacted, establishes requirements for issua...",2017-2018 Regular Session,passed,"Education, Higher",https://legiscan.com/TN/bill/SB2174/2017,Education,Yea,94.0,0.0,94.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189618,SB0648,B,S,"As enacted, allocates a portion of sales and u...",2019-2020 Regular Session,passed,"Taxes, Sales",https://legiscan.com/TN/bill/SB0648/2019,Taxes,Yea,94.0,0.0,95.0,False
1180616,HB0294,B,H,"As introduced, allows, subject to available ap...",2019-2020 Regular Session,introduced,Education,https://legiscan.com/TN/bill/HB0294/2019,Education,no record,no record,no record,no record,False
1332143,SB2675,B,S,"As introduced, changes the date, from August 1...",2019-2020 Regular Session,introduced,Education,https://legiscan.com/TN/bill/SB2675/2019,Education,no record,no record,no record,no record,False
